# 2D Annular Shear of peanuts

## Creating sample

In [ ]:
from pathlib import Path

import numpy as np
import math

from pylmgc90 import pre

In [ ]:
def rigidPeanuts(r, center, model, material, color):
    import random 
    bd = pre.avatar(dimension=2)
    bd.addNode( pre.node(coor=np.array(center),number=1) )
    bd.addBulk( pre.rigid2d() )
    bd.defineGroups()
    bd.defineModel(model=model)
    bd.defineMaterial(material=material)
    bd.addContactors(shape='DISKx', color=color, byrd=r*0.5,shift=[-r*0.5,0.])
    bd.addContactors(shape='DISKx', color=color, byrd=r*0.5,shift=[r*0.5,0.]) 
    bd.computeRigidProperties()
    bd.rotate(description='axis', alpha=math.pi*random.random(), axis=[0., 0., 1.], center=bd.nodes[1].coor) 

    return bd

In [ ]:
datbox = Path('./DATBOX')
datbox.mkdir(exist_ok=True)

# 2D
dim = 2

# containers
#   * bodies
bodies = pre.avatars()
#   * materials
mats = pre.materials()
#   * models
mods = pre.models()
#   * visibility tables
svs = pre.see_tables()
#   * contact laws
tacts = pre.tact_behavs()

# materials
tdur = pre.material(name='TDURx',materialType='RIGID',density=1000.)
plex = pre.material(name='PLEXx',materialType='RIGID',density=100.)
mats.addMaterial(tdur,plex)

# rigid model creation
mod = pre.model(name='rigid', physics='MECAx', element='Rxx2D', dimension=dim)

# 2000 particles
nb_particles=2000

# random distribution [0.5, 2.[ 
radii = pre.granulo_Random(nb_particles, 0.5, 2.)

# 
radius_min = np.amin(radii)
radius_max = np.amax(radii)

# deposit in Couette
rint = 20. 
rext = 50.
nb_remaining_particles, coor, radii = pre.depositInCouette2D(radii, rint, rext)
# warning
if (nb_remaining_particles < nb_particles):
    print("Warning: original granulometry changed since all particles are not kept!")

# adding bodies :
for r, c in zip(radii, coor):
    body = rigidPeanuts(r=r, center=c, model=mod, material=plex, color='BLUEx') 
    bodies += body

# inner and outer cylinders:
inner = pre.rigidDisk(r=rint, center=[rext, rext], model=mod, material=tdur, color='WALLx')
outer = pre.rigidDisk(r=rext, center=[rext, rext], model=mod, material=tdur, color='WALLx', is_Hollow=True)
bodies += inner; bodies += outer

# boundary conditions:
inner.imposeDrivenDof(component=[1, 2],dofty='vlocy')
inner.imposeDrivenDof(component=3,dofty='vlocy',description='evolution',evolutionFile='wz.txt')

ofile = open('./DATBOX/wz.txt','w')
ofile.write('%12.5e %12.5e\n' % (0.,0.))
ofile.write('%12.5e %12.5e\n' % (5.,0.))
ofile.write('%12.5e %12.5e\n' % (6.,0.1))
ofile.write('%12.5e %12.5e\n' % (16.,0.1))
ofile.write('%12.5e %12.5e\n' % (17.,0.))
ofile.write('%12.5e %12.5e\n' % (18.,0.))
ofile.close()

outer.imposeDrivenDof(component=[1, 2, 3], dofty='vlocy')

# contact laws
ldkdk = pre.tact_behav(name='iqsc0',law='IQS_CLB',fric=0.3)
tacts+= ldkdk

ldkkd = pre.tact_behav(name='iqsc1',law='IQS_CLB',fric=0.5)
tacts+= ldkkd

# visibility tables
svdkdk = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='DISKx', colorCandidat   ='BLUEx',
                       CorpsAntagoniste='RBDY2', antagoniste='DISKx', colorAntagoniste='BLUEx',
                       behav=ldkdk, alert=0.1*radius_min)
svs += svdkdk

svdkdk_inner = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='DISKx', colorCandidat   ='BLUEx',
                             CorpsAntagoniste='RBDY2', antagoniste='DISKx', colorAntagoniste='WALLx',
                             behav=ldkkd, alert=0.1*radius_min)
svs+=svdkdk_inner

svdkkd = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='DISKx', colorCandidat='BLUEx',
                       CorpsAntagoniste='RBDY2', antagoniste='xKSID', colorAntagoniste='WALLx',
                       behav=ldkkd, alert=0.1*radius_min)
svs += svdkkd


post = pre.postpro_commands()

post.addCommand( pre.postpro_command(name='SOLVER INFORMATIONS', step=1) )
post.addCommand( pre.postpro_command(name='TORQUE EVOLUTION', step=1,rigid_set=[inner] ) )
post.addCommand( pre.postpro_command(name='BODY TRACKING', step=1,rigid_set=[inner] ) )


# ecriture des fichiers
pre.writeDatbox(dim, mats, mods, bodies, tacts, svs, post=post, datbox_path=datbox, gravy=[0., 0., 0.])

In [ ]:
try:
    pre.visuAvatars(bodies)
except:
    pass

## Computation

In [ ]:
import math
import numpy as np

from pylmgc90 import chipy
from pylmgc90.chipy import computation

# Initializing
chipy.Initialize()

# checking/creating mandatory subfolders
chipy.checkDirectories()

# logMes
chipy.utilities_DisableLogMes()


# driving radius expansion
#
H=1e-2
# the given pressure 0.1MPa
P=1e5
# pseudo mass
M=5e4
# max radial velocity amplitude
Vdmax=H*P/M
# needed by algorithm
Vd=0.
drdila = 0.
Pres=0.
Pi=0.
#
# defining some variables
#

# space dimension
dim = 2

# modeling hypothesis ( 1 = plain strain, 2 = plain stress, 3 = axi-symmetry)
mhyp = 1

# time evolution parameters
dt = H
nb_steps = 500

# theta integrator parameter
theta = 0.5

# deformable  yes=1, no=0
deformable = 0

# interaction parameters
Rloc_tol = 5.e-2

# nlgs parameters
tol = 1e-6
relax = 1.0
norm = 'Quad '
gs_it1 = 150
gs_it2 = 30
solver_type='Stored_Delassus_Loops         '

# write parameter
freq_write   = 10

# display parameters
freq_display = 10

chipy.nlgs_SetWithQuickScramble()

computation.initialize(dim, dt, theta)

nb_DISKx = chipy.DISKx_GetNbDISKx()
# to store results
evol_pres=np.zeros((4,nb_steps))

nbt=6
v_tranches=np.zeros((nb_steps,nbt))
lrext=np.zeros(nb_steps)

#
#
# simulation part ...
#

for k in range(1,nb_steps+1):
    
    if k%50 == 0:
        print( f"computing step {k}" )

    #
    chipy.utilities_logMes('INCREMENT STEP')

    chipy.utilities_EnableLogMes()
    chipy.IncrementStep()
    chipy.utilities_DisableLogMes()

    chipy.utilities_logMes('COMPUTE Fext')
    chipy.ComputeFext()
    chipy.utilities_logMes('COMPUTE Fint')
    chipy.ComputeBulk()
    chipy.utilities_logMes('COMPUTE Free Vlocy')
    chipy.ComputeFreeVelocity()

    # correction de la vitesse (M*(Vd-Vdi)=dt*(Pres -P) => Vd = Vdi + dt*(Pres-P)/M)    
    Vi = Vd       
    Vd+= dt*(Pres-P)/M
    
    chipy.xKSID_SetVdilation(1,Vd)

    chipy.utilities_logMes('SELECT PROX TACTORS')
    chipy.SelectProxTactors()

    chipy.utilities_logMes('RESOLUTION' )
    chipy.RecupRloc(Rloc_tol)

    chipy.ExSolver(solver_type, norm, tol, relax, gs_it1, gs_it2)
    chipy.UpdateTactBehav()

    chipy.StockRloc()

    chipy.utilities_logMes('COMPUTE DOF, FIELDS, etc.')
    chipy.ComputeDof()

    chipy.utilities_logMes('UPDATE DOF, FIELDS')
    chipy.UpdateStep()

    # decreasing particle radius
    drdila += dt*0.5*(Vd+Vi)
    chipy.xKSID_SetXdilation(1,drdila)

    chipy.utilities_logMes('WRITE OUT DOF')
    chipy.WriteOutDof(freq_write)
    chipy.utilities_logMes('WRITE OUT Rloc')
    chipy.WriteOutVlocRloc(freq_write)

    chipy.utilities_logMes('VISU & POSTPRO')
    chipy.WriteDisplayFiles(freq_display)
    chipy.WritePostproFiles()

    # computing pressure on hollow cylinder
    Pi = Pres
    Pres = 0.
    R = chipy.xKSID_GetContactorRadius(1)
    inters = chipy.getInteractions()
    inters_dkkd = inters[ inters['inter']==b'DKKDx' ]
    Pres = np.sum( inters_dkkd['rl'][:,1] )
    Pres/=2*math.pi*R

    evol_pres[0,k-1]=chipy.TimeEvolution_GetTime()
    evol_pres[1,k-1]=Vd
    evol_pres[2,k-1]=R
    evol_pres[3,k-1]=Pres    

    # computing mean velocity along a radius
    nbr=chipy.RBDY2_GetNbRBDY2()
    nbd =chipy.DISKx_GetNbDISKx()
    rint=chipy.DISKx_GetContactorRadius(nbd)
    rext=chipy.xKSID_GetContactorRadius(1)
    center=chipy.RBDY2_GetBodyVector('Coor_', nbr-1)

    lrext[k-1]=rext
    # thickness of a slab
    dr=(rext-rint)/nbt
  
    # loop on disks (boundaries are the two last objects)
    coor = chipy.RBDY2_GetAllBodyVector('Coor_')
    coor = coor[:-2,:2]

    # radius
    coor = coor-center[:2]
    radius = np.linalg.norm(coor,axis=1)
    # radius = radius if radius<rext else rext
    radius = np.where(radius<rext,radius,rext)

    # orientation
    anr = coor/radius[:,np.newaxis]
    atr = anr[:,[1,0]]
    atr[:,0] *= -1

    # velocity
    velo = chipy.RBDY2_GetAllBodyVector('V____')
    velo = velo[:-2,:2]

    # orthoradial component
    vorth = np.sum(velo[:,:]*atr[:,:],axis=1)
    vorth = np.einsum('ij,ij->i', velo, atr)

    # sort by slice
    it = np.floor((radius-rint)/dr).astype(int)
    v_tranches[k-1,:] = np.bincount(it, weights=vorth)
    alit, nbpt = np.unique(it, return_counts=True)
    v_tranches[k-1,alit] /= nbpt

#
# close display & postpro
#

computation.finalize()

In [ ]:
import pickle

with open("pres.p", "wb" ) as f:
    pickle.dump(evol_pres, f )

with open("rext.p", "wb" ) as f:
    pickle.dump(lrext, f )

with open("velocity.p", "wb" ) as f:
    pickle.dump(v_tranches, f )


In [ ]:
import matplotlib.pyplot as plt
plt.subplot(311)
plt.plot(evol_pres[0],evol_pres[3])
plt.xlabel('time')
plt.ylabel('pressure')
plt.subplot(312)
plt.plot(evol_pres[0],evol_pres[2])
plt.xlabel('time')
plt.ylabel('radius')
plt.subplot(313)
plt.plot(evol_pres[0],evol_pres[1])
plt.xlabel('time')
plt.ylabel('velocity')
plt.show()

r=np.linspace(rint,lrext[-1],nbt)
plt.plot(r,v_tranches[-1,:])
plt.xlabel('radius')
plt.ylabel('mean velocity')
plt.show()